In [2]:
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import random
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf


Descarga de datos

In [3]:
!wget https://www.dropbox.com/s/57tel5zqopkssrh/books.csv?dl=0 -O books.csv
!wget https://www.dropbox.com/s/zpnnoy1i8ljf9fg/goodreads_bert_embeddings.npy?dl=0 -O goodreads_bert_embeddings.npy
!wget https://www.dropbox.com/s/a8hcc9w30y7r3jl/goodreads_bert_large_embeddings.npy?dl=0 -O goodreads_bert_large_embeddings.npy
!wget https://www.dropbox.com/s/dqeqpsr0vdvmcy0/goodreads_past_interactions.json?dl=0 -O goodreads_past_interactions.json
!wget https://www.dropbox.com/s/rjtzhmb2zbpp30q/goodreads_test_interactions.json?dl=0 -O goodreads_test_interactions.json

--2025-07-01 02:44:37--  https://www.dropbox.com/s/57tel5zqopkssrh/books.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/5s6xrfnu17yi34sfhmskb/books.csv?rlkey=ymzokbyqw3qq2bq5okfao9w1z&dl=0 [following]
--2025-07-01 02:44:37--  https://www.dropbox.com/scl/fi/5s6xrfnu17yi34sfhmskb/books.csv?rlkey=ymzokbyqw3qq2bq5okfao9w1z&dl=0
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd330380380df4aa50453d4ab76.dl.dropboxusercontent.com/cd/0/inline/CspXdKc1NsHjy5xXdEc8jHVKrp8XwnL1H7xRVhGseerOvZ6yoSmhkRnUdFUEYs68dC_v8KZsKqCyxdL0jTt6CsMisZIPqGlb-a1IT3LMITMjX6uu5ymHAjMVh_jC4TH5ZXayYmdHJnC0K5NXQzrueG5j/file# [following]
--2025-07-01 02:44:37--  https://ucd330380380df4aa50453d4ab76.dl.dropboxusercontent.com/cd/0/in

In [4]:
df_books = pd.read_csv('books.csv')
df_books.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,book_desc
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,Winning will make you famous. Losing means cer...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,Harry Potter's life is miserable. His parents ...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...,About three things I was absolutely positive.F...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...,The unforgettable novel of a childhood in a sl...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...,Alternate Cover Edition ISBN: 0743273567 (ISBN...


In [5]:
with open('goodreads_past_interactions.json', 'r') as f:
    user_interactions = json.load(f)


In [6]:
with open('goodreads_test_interactions.json', 'r') as f:
    user_interactions_test = json.load(f)

In [7]:
idx2userid = {i: id_ for i, id_ in enumerate(user_interactions.keys())}
userid2idx = {id_:i for i, id_ in enumerate(user_interactions.keys())}

In [8]:
# 2) Build userid2idx by enumerating all distinct user IDs in the training set.
#    (You could also include users from test if there are any users in test not in train,
#     but typically we drop test-users that never appear in train.)

userid2idx = { raw_uid: idx
               for idx, raw_uid in enumerate(user_interactions.keys()) }

#    If you want to ensure we include any “new” users that appear only in test:
for raw_uid in user_interactions_test.keys():
    if raw_uid not in userid2idx:
        userid2idx[raw_uid] = len(userid2idx)

In [9]:
# 3) Build the set of all book IDs that appear in either train or test:
all_book_ids = set()

#    From train:
for raw_uid, book_list in user_interactions.items():
    for raw_bid in book_list:
        all_book_ids.add(raw_bid)

#    From test:
for raw_uid, book_list in user_interactions_test.items():
    for raw_bid in book_list:
        all_book_ids.add(raw_bid)

In [10]:
# 4) Now enumerate that set to build bookid2idx
bookid2idx = { raw_bid: idx for idx, raw_bid in enumerate(all_book_ids) }

In [11]:
# 5) Now you can check:
num_users = len(userid2idx)
num_items = len(bookid2idx)

print(f"Number of distinct users: {num_users}")
print(f"Number of distinct books: {num_items}")

Number of distinct users: 52821
Number of distinct books: 4287


In [12]:
unique_books = df_books.shape[0]
num_columns_table = df_books.shape[1]

print(f"Number of unique books: {unique_books}")
print(f"Number of columns in the table: {num_columns_table}")

Number of unique books: 4287
Number of columns in the table: 24


## Build “(user, item, label)” arrays for training

First, we’ll construct a dictionary of “positive item-indices per user_idx” for training:

In [13]:
from collections import defaultdict

# pos_items_per_user[u_idx] = set of i_idx that user u_idx interacted with in train
pos_items_per_user = defaultdict(set)

for raw_u, book_list in user_interactions.items():
    u_idx = userid2idx[raw_u]
    for raw_b in book_list:
        i_idx = bookid2idx[raw_b]
        pos_items_per_user[u_idx].add(i_idx)

# (Now pos_items_per_user[u_idx] is a Python set of i_idx for each user)

Next, negative sampling. For each positive pair (u_idx, i_idx), we will sample n_neg_per_pos “negatives” j ∈ [0..num_items−1] such that j ∉ pos_items_per_user[u_idx]. Let’s choose 4 negatives per positive:

In [14]:
n_neg_per_pos = 2
rng = np.random.default_rng(seed=42)

user_input = []
item_input = []
labels = []

for u_idx, pos_set in pos_items_per_user.items():
    # If a user has no positives (unlikely), skip.
    if len(pos_set) == 0:
        continue

    for i_idx in pos_set:
        # — Positive sample —
        user_input.append(u_idx)
        item_input.append(i_idx)
        labels.append(1.0)

        # — Negative samples —
        neg_count = 0
        while neg_count < n_neg_per_pos:
            sampled_i = rng.integers(low=0, high=num_items)
            if sampled_i not in pos_set:
                user_input.append(u_idx)
                item_input.append(sampled_i)
                labels.append(0.0)
                neg_count += 1

# Convert lists → NumPy arrays for Keras
user_input = np.array(user_input, dtype=np.int32)   # shape = (N_total_examples,)
item_input = np.array(item_input, dtype=np.int32)   # shape = (N_total_examples,)
labels     = np.array(labels,     dtype=np.float32) # shape = (N_total_examples,)

print("train triples:", user_input.shape, item_input.shape, labels.shape)


train triples: (10063569,) (10063569,) (10063569,)


## Build (user, item, label) arrays for validation (test)

In [15]:
# 1) Construct pos_items_per_user_test[u_idx]
pos_items_per_user_test = defaultdict(set)
for raw_u, book_list in user_interactions_test.items():
    # It’s possible that user_interactions_test contains a user not seen in training.
    # If so, skip them (or add them to userid2idx + pos_items_per_user if you want cold-start eval).
    if raw_u not in userid2idx:
        continue
    u_idx = userid2idx[raw_u]
    for raw_b in book_list:
        if raw_b not in bookid2idx:
            # skip books that weren’t in your mapping
            continue
        i_idx = bookid2idx[raw_b]
        pos_items_per_user_test[u_idx].add(i_idx)

# 2) Negative sampling for test:
user_input_test = []
item_input_test = []
labels_test = []

for u_idx, pos_set in pos_items_per_user_test.items():
    if len(pos_set) == 0:
        continue

    for i_idx in pos_set:
        # Positive
        user_input_test.append(u_idx)
        item_input_test.append(i_idx)
        labels_test.append(1.0)

        # Negatives
        neg_count = 0
        while neg_count < n_neg_per_pos:
            sampled_i = rng.integers(low=0, high=num_items)
            if sampled_i not in pos_set:
                user_input_test.append(u_idx)
                item_input_test.append(sampled_i)
                labels_test.append(0.0)
                neg_count += 1

user_input_test = np.array(user_input_test, dtype=np.int32)
item_input_test = np.array(item_input_test, dtype=np.int32)
labels_test     = np.array(labels_test,     dtype=np.float32)

print("test triples:", user_input_test.shape, item_input_test.shape, labels_test.shape)

test triples: (3000,) (3000,) (3000,)


## Item features

In [16]:
bert_embeddings = np.load("goodreads_bert_embeddings.npy")  # shape: (num_items, embedding_dim)
#bert_embeddings_large = np.load("goodreads_bert_large_embeddings.npy")

num_items, embedding_dim = bert_embeddings.shape

print(bert_embeddings.shape)


(4287, 768)


In [17]:
# For training
#item_feat_input = item_features[item_input]         # shape = (N_train, D)
#item_feat_input = bert_embeddings[item_input]
# For validation
#item_feat_input_test = item_features[item_input_test]
#item_feat_input_test = bert_embeddings[item_input_test]

# Functions for Ranking Metrics

## Metricas de precision

In [18]:
import numpy as np

def precision_at_k(predicted_items, true_set, k):
    """
    predicted_items: list or 1D array of item-indices ranked by descending score (length ≥ k)
    true_set: a Python set of ground-truth positive items for this user
    k: cutoff
    """
    pred_k = predicted_items[:k]
    hits = sum((1 for i in pred_k if i in true_set))
    return hits / k

def dcg_at_k(predicted_items, true_set, k):
    """
    DCG@k = sum_{i=1..k} ( rel_i / log2(i+1) ), where rel_i = 1 if predicted_items[i-1] ∈ true_set
    """
    dcg = 0.0
    for rank, item in enumerate(predicted_items[:k], start=1):
        if item in true_set:
            dcg += 1.0 / np.log2(rank + 1)
    return dcg

def idcg_at_k(true_set, k):
    """
    Ideal DCG@k for this user, i.e. if we ranked all true items first.
    IDCG = sum_{i=1..min(len(true_set), k)} (1 / log2(i+1))
    """
    n_rel = min(len(true_set), k)
    if n_rel == 0:
        return 0.0
    return sum((1.0 / np.log2(i + 1) for i in range(1, n_rel + 1)))

def ndcg_at_k(predicted_items, true_set, k):
    """
    NDCG@k = DCG@k / IDCG@k
    """
    dcg = dcg_at_k(predicted_items, true_set, k)
    idcg = idcg_at_k(true_set, k)
    return 0.0 if idcg == 0 else dcg / idcg

def average_precision_at_k(predicted_items, true_set, k):
    """
    AP@k for a single user:
    AP = (1 / min(n_pos, k)) * sum_{i=1..k} ( precision@i * rel_i ),
    where rel_i = 1 if predicted_items[i-1] ∈ true_set, else 0.
    If the user has no positives, define AP = 0.0.
    """
    if len(true_set) == 0:
        return 0.0

    hits = 0
    score = 0.0
    for idx, item in enumerate(predicted_items[:k], start=1):
        if item in true_set:
            hits += 1
            score += hits / idx  # precision@idx
    return score / min(len(true_set), k)


Prepare per‐user ground‐truth sets

In [19]:
from collections import defaultdict

# 1) Build train_pos (you probably have this as pos_items_per_user):
train_pos = defaultdict(set)
for raw_u, book_list in user_interactions.items():
    u_idx = userid2idx[raw_u]
    for raw_b in book_list:
        train_pos[u_idx].add(bookid2idx[raw_b])

# 2) Build test_pos similarly:
test_pos = defaultdict(set)
for raw_u, book_list in user_interactions_test.items():
    if raw_u not in userid2idx:
        continue
    u_idx = userid2idx[raw_u]
    for raw_b in book_list:
        if raw_b not in bookid2idx:
            continue
        test_pos[u_idx].add(bookid2idx[raw_b])


In [20]:
all_users = list(test_pos.keys())  # only users with ≥1 test interaction
num_items_check = len(bookid2idx)        # total number of distinct items


## Metricas de diversidad y novelty

Contador de Popularidad en Train

In [21]:
from collections import Counter
all_train_pairs = [
    (u_idx, i_idx)
    for u_idx, books in user_interactions.items()
    for i_idx in [bookid2idx[b] for b in books]
]
pop_counter = Counter(i for _, i in all_train_pairs)
max_pop = max(pop_counter.values())

Funciones

In [22]:
from sklearn.metrics.pairwise import cosine_similarity

# Intra list diversity
def ild_at_k(topk, feat_matrix, k):
    vecs = feat_matrix[topk[:k]]              # (k, D)
    sims = cosine_similarity(vecs)            # (k, k)
    i, j = np.triu_indices(k, k=1)
    return np.mean(1.0 - sims[i, j])

# Novelty basado en popularidad global
def novelty_popularity(topk, pop_counter, max_pop, k):
    pops = np.array([pop_counter[i] for i in topk[:k]], dtype=np.float32)
    return np.mean(1.0 - pops / max_pop)

## Custom Keras callback to compute metrics

In [23]:
import tensorflow as tf

class RankingMetricsCallback(tf.keras.callbacks.Callback):
    def __init__(self, train_pos, test_pos, num_items, k=10, verbose=1):
        """
        train_pos: dict[u_idx] -> set of item indices the user interacted with in training
        test_pos: dict[u_idx] -> set of item indices for ground-truth test positives
        num_items: total number of distinct items
        k: cutoff for precision@k, NDCG@k, MAP@k
        verbose: 0 or 1 (whether to print results)
        """
        super().__init__()
        self.train_pos = train_pos
        self.test_pos = test_pos
        self.num_items = num_items
        self.k = k
        self.verbose = verbose

        # Precompute for speed: a full array of all item indices
        self.all_items_arr = np.arange(num_items, dtype=np.int32)

    def on_epoch_end(self, epoch, logs=None):
        """
        At the end of each epoch, compute average Precision@k, NDCG@k, MAP@k over all users
        in test_pos. Adds metrics to logs dictionary and prints them.
        """
        precision_list = []
        ndcg_list = []
        ap_list = []
        ilds = []
        novelties = []

        # For each user in test_pos:
        for u_idx, true_set in self.test_pos.items():
            if not true_set:
                continue

            # 1) Build candidate items = all_items \ train_pos[u]
            train_set = self.train_pos.get(u_idx, set())
            # Boolean mask for candidates:
            if len(train_set) == 0:
                candidates = self.all_items_arr
            else:
                mask = np.ones(self.num_items, dtype=bool)
                mask[list(train_set)] = False
                candidates = self.all_items_arr[mask]

            # 2) Build user and item batches for prediction
            u_batch = np.full(shape=(candidates.shape[0],), fill_value=u_idx, dtype=np.int32)
            i_batch = candidates

            # 3) Get scores from the model
            #    Note: model.predict(...) can be batched inside (it handles large arrays)
            scores = self.model.predict([u_batch, i_batch], batch_size=2048, verbose=0).flatten()

            # 4) Get top-k items by descending score
            topk_indices = np.argpartition(-scores, self.k - 1)[:self.k]
            # `argpartition` is O(n), then we sort those k for final ranking:
            topk_scores = scores[topk_indices]
            topk_items = candidates[topk_indices][np.argsort(-topk_scores)]

            # 5) Compute metrics for this user
            p_at_k = precision_at_k(topk_items, true_set, self.k)
            ndcg_k = ndcg_at_k(topk_items, true_set, self.k)
            ap_k = average_precision_at_k(topk_items, true_set, self.k)

            ild_k = ild_at_k(topk_items, bert_embeddings, self.k)
            novelty_k = novelty_popularity(topk_items, pop_counter, max_pop, self.k)

            precision_list.append(p_at_k)
            ndcg_list.append(ndcg_k)
            ap_list.append(ap_k)

            ilds.append(ild_k)
            novelties.append(novelty_k)

        # 6) Average across all users
        avg_precision = float(np.mean(precision_list)) if precision_list else 0.0
        avg_ndcg      = float(np.mean(ndcg_list))      if ndcg_list else 0.0
        avg_map       = float(np.mean(ap_list))        if ap_list else 0.0
        avg_ild       = float(np.mean(ilds))          if ilds else 0.0
        avg_novelty   = float(np.mean(novelties))     if novelties else 0.0

        # 7) Log into `logs` so Keras will record them
        if logs is not None:
            logs[f"Precision@{self.k}"] = avg_precision
            logs[f"NDCG@{self.k}"]      = avg_ndcg
            logs[f"MAP@{self.k}"]       = avg_map
            logs[f"ILD@{self.k}"]      = avg_ild
            logs[f"Novelty@{self.k}"]   = avg_novelty

        # 8) Optionally print to console
        if self.verbose > 0:
            print(
                f" — epoch {epoch + 1:>2d}: "
                f"Precision@{self.k}={avg_precision:.4f}, "
                f"NDCG@{self.k}={avg_ndcg:.4f}, "
                f"MAP@{self.k}={avg_map:.4f}"
                f"ILD@{self.k}={avg_ild:.4f}"
                f"Novelty@{self.k}={avg_novelty:.4f}"
            )


In [24]:
import tensorflow as tf

class MultiKRankingCallback(tf.keras.callbacks.Callback):
    def __init__(self, train_pos, test_pos, num_items, Ks=[5,10,20], verbose=1):
        """
        train_pos: dict[u_idx] -> set of item indices the user interacted with in training
        test_pos: dict[u_idx] -> set of item indices for ground-truth test positives
        num_items: total number of distinct items
        k: cutoff for precision@k, NDCG@k, MAP@k
        verbose: 0 or 1 (whether to print results)
        """
        super().__init__()
        self.train_pos = train_pos
        self.test_pos = test_pos
        self.num_items = num_items
        self.Ks = Ks
        self.verbose = verbose

        # Precompute for speed: a full array of all item indices
        self.all_items_arr = np.arange(num_items, dtype=np.int32)

    def on_epoch_end(self, epoch, logs=None):
        """
        At the end of each epoch, compute average Precision@k, NDCG@k, MAP@k over all users
        in test_pos. Adds metrics to logs dictionary and prints them.
        """
        precision_list = []
        ndcg_list = []
        ap_list = []
        ilds = []
        novelties = []

        metrics = {K: {"prec":[], "ndcg":[], "map":[],"ild":[], "nov":[]} for K in self.Ks}

        # For each user in test_pos:
        for u_idx, true_set in self.test_pos.items():
            if not true_set:
                continue

            # 1) Build candidate items = all_items \ train_pos[u]
            train_set = self.train_pos.get(u_idx, set())
            # Boolean mask for candidates:
            if len(train_set) == 0:
                candidates = self.all_items_arr
            else:
                mask = np.ones(self.num_items, dtype=bool)
                mask[list(train_set)] = False
                candidates = self.all_items_arr[mask]

            # 2) Build user and item batches for prediction
            u_batch = np.full(shape=(candidates.shape[0],), fill_value=u_idx, dtype=np.int32)
            i_batch = candidates

            # 3) Get scores from the model
            #    Note: model.predict(...) can be batched inside (it handles large arrays)
            scores = self.model.predict([u_batch, i_batch], batch_size=2048, verbose=0).flatten()

            for K in self.Ks:
                # 4) Get top-k items by descending score
                topk_indices = np.argpartition(-scores, K - 1)[:K]
                # `argpartition` is O(n), then we sort those k for final ranking:
                topk_scores = scores[topk_indices]
                topk_items = candidates[topk_indices][np.argsort(-topk_scores)]

                # 5) Compute metrics for this user
                p_at_k = precision_at_k(topk_items, true_set, K)
                ndcg_k = ndcg_at_k(topk_items, true_set, K)
                ap_k = average_precision_at_k(topk_items, true_set, K)

                ild_k = ild_at_k(topk_items, bert_embeddings, K)
                novelty_k = novelty_popularity(topk_items, pop_counter, max_pop, K)

                m = metrics[K]
                m["prec"].append(p_at_k)
                m["ndcg"].append(ndcg_k)
                m["map"].append(ap_k)
                m["ild"].append(ild_k)
                m["nov"].append(novelty_k)

        # 6) Average across all users and log so Keras will record them
        for K in self.Ks:
            m = metrics[K]
            logs[f"Precision@{K}"] = float(np.mean(m["prec"])) if m["prec"] else 0.0
            logs[f"NDCG@{K}"]      = float(np.mean(m["ndcg"])) if m["ndcg"] else 0.0
            logs[f"MAP@{K}"]       = float(np.mean(m["map"])) if m["map"] else 0.0
            logs[f"ILD@{K}"]      = float(np.mean(m["ild"])) if m["ild"] else 0.0
            logs[f"Novelty@{K}"]   = float(np.mean(m["nov"])) if m["nov"] else 0.0

        # 8) Optionally print to console
        if self.verbose > 0:
            line = f" — epoch {epoch + 1}: "
            for K in self.Ks:
                m = metrics[K]
                line += (
                    f"  P@{K}={logs[f'Precision@{K}']:.3f},"
                    f" NDCG@{K}={logs[f'NDCG@{K}']:.3f},"
                    f" MAP@{K}={logs[f'MAP@{K}']:.3f},"
                    f" ILD@{K}={logs[f'ILD@{K}']:.3f},"
                    f" Nov@{K}={logs[f'Novelty@{K}']:.3f};"
                )
            print(line)


# Vanilla NCF (Keras)


In [25]:
from tensorflow.keras.layers import Input, Embedding, Flatten, Multiply, Concatenate, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")

# 1) Reduced hyperparameters
K_mf  = 16
K_mlp = 32
mlp_layers = [32, 16]
dropout_rate = 0.2

# 2) Define inputs
user_in = Input(shape=(), dtype=tf.int32, name="user_input")
item_in = Input(shape=(), dtype=tf.int32, name="item_input")

# 3) GMF branch
u_mf = Embedding(num_users, K_mf, name="u_embed_mf")(user_in)
i_mf = Embedding(num_items, K_mf, name="i_embed_mf")(item_in)
u_latent_mf = Flatten()(u_mf)
i_latent_mf = Flatten()(i_mf)
mf_vec = Multiply()([u_latent_mf, i_latent_mf])  # (batch, K_mf)

# 4) MLP branch (smaller dims)
u_mlp_emb = Embedding(num_users, K_mlp, name="u_embed_mlp")(user_in)
i_mlp_emb = Embedding(num_items, K_mlp, name="i_embed_mlp")(item_in)
u_latent_mlp = Flatten()(u_mlp_emb)
i_latent_mlp = Flatten()(i_mlp_emb)

mlp_vec = Concatenate()([u_latent_mlp, i_latent_mlp])  # (batch, 2*K_mlp)
for idx, size in enumerate(mlp_layers):
    mlp_vec = Dense(units=size, activation="relu", name=f"mlp_dense_{idx}")(mlp_vec)
    mlp_vec = Dropout(dropout_rate)(mlp_vec)

# 5) Combine GMF + MLP
concat = Concatenate()([mf_vec, mlp_vec])
output = Dense(units=1, activation="sigmoid", name="prediction")(concat)

model_vanilla = Model(inputs=[user_in, item_in], outputs=output)
model_vanilla.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=[tf.keras.metrics.AUC(name="auc")]
)
model_vanilla.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_input          │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ item_input          │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ u_embed_mlp         │ (None, 32)        │  1,690,272 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ i_embed_mlp         │ (None, 32)        │    137,184 │ item_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_2 (Flatten) │ (None, 32)        │          0 │ u_embed_mlp[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_3 (Flatten) │ (None, 32)        │          0 │ i_embed_mlp[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 64)        │          0 │ flatten_2[0][0],  │
│ (Concatenate)       │                   │            │ flatten_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_dense_0 (Dense) │ (None, 32)        │      2,080 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ u_embed_mf          │ (None, 16)        │    845,136 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ i_embed_mf          │ (None, 16)        │     68,592 │ item_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 32)        │          0 │ mlp_dense_0[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 16)        │          0 │ u_embed_mf[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 16)        │          0 │ i_embed_mf[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_dense_1 (Dense) │ (None, 16)        │        528 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 16)        │          0 │ flatten[0][0],    │
│                     │                   │            │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 16)        │          0 │ mlp_dense_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 32)        │          0 │ multiply[0][0],   │
│ (Concatenate)       │                   │            │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ prediction (Dense)  │ (None, 1)         │         33 │ concatenate_1[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,743,825 (10.47 MB)

 Trainable params: 2,743,825 (10.47 MB)

 Non-trainable params: 0 (0.00 B)

Before training, prepare callback for metrics

In [26]:
ks = [5, 10, 20]
rank_cb = MultiKRankingCallback(
    train_pos=train_pos,      # user → set of train-positive items
    test_pos=test_pos,        # user → set of test-positive items
    num_items=num_items,      # total number of distinct items (4287)
    Ks=ks,                     # compute Precision@10, NDCG@10, MAP@10
    verbose=1                 # set to 1 to print metrics each epoch
)

Train on tf.data.Dataset


In [27]:
batch_size = 4096  # increase if your GPU/Colab can handle it (maybe 4096)
buffer_size = 100_000

# 1) Build a Dataset from your three 1D NumPy arrays
#    Each element is (u_idx, i_idx, label).
train_ds = tf.data.Dataset.from_tensor_slices((user_input, item_input, labels))

# 2) Shuffle → batch → prefetch.  Drop remainder so each batch is exactly `batch_size`.
train_ds = (
    train_ds
    .shuffle(buffer_size, seed=42)
    .batch(batch_size, drop_remainder=True)
    .map(lambda u, i, y: ((u, i), y), num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(tf.data.AUTOTUNE)
)

# 3) Do the same for validation (but typically you skip shuffle for val):
val_ds = tf.data.Dataset.from_tensor_slices(
    (user_input_test, item_input_test, labels_test)
)
val_ds = (
    val_ds
    .batch(batch_size, drop_remainder=False)
    .map(lambda u, i, y: ((u, i), y), num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(tf.data.AUTOTUNE)
)

# 4) Now call .fit on the Datasets:
history_vanilla = model_vanilla.fit(
    train_ds,
    epochs=5,
    validation_data=val_ds,
    callbacks=[rank_cb],
    shuffle=False  # we already shuffled in the Dataset pipeline
)


Epoch 1/5
2456/2456 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - auc: 0.8330 - loss: 0.4623 — epoch 1:   P@5=0.116, NDCG@5=0.126, MAP@5=0.070, ILD@5=0.313, Nov@5=0.512;  P@10=0.093, NDCG@10=0.108, MAP@10=0.045, ILD@10=0.308, Nov@10=0.569;  P@20=0.083, NDCG@20=0.147, MAP@20=0.057, ILD@20=0.302, Nov@20=0.643;
2456/2456 ━━━━━━━━━━━━━━━━━━━━ 173s 69ms/step - auc: 0.8330 - loss: 0.4623 - val_auc: 0.9168 - val_loss: 0.3541 - Precision@5: 0.1160 - NDCG@5: 0.1260 - MAP@5: 0.0697 - ILD@5: 0.3130 - Novelty@5: 0.5115 - Precision@10: 0.0930 - NDCG@10: 0.1081 - MAP@10: 0.0448 - ILD@10: 0.3080 - Novelty@10: 0.5686 - Precision@20: 0.0825 - NDCG@20: 0.1474 - MAP@20: 0.0566 - ILD@20: 0.3019 - Novelty@20: 0.6427
Epoch 2/5
2456/2456 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - auc: 0.9140 - loss: 0.3484 — epoch 2:   P@5=0.116, NDCG@5=0.128, MAP@5=0.071, ILD@5=0.305, Nov@5=0.541;  P@10=0.113, NDCG@10=0.122, MAP@10=0.049, ILD@10=0.303, Nov@10=0.590;  P@20=0.099, NDCG@20=0.168, MAP@20=0.065, ILD@20=0.303, Nov@20=0.653;
2456/

# NCF + Extra Embeddings

In [28]:
import numpy as np

# Suppose bert_embeddings is a NumPy array of shape (num_items, D)
bert_embeddings = np.load("goodreads_bert_embeddings.npy")  # (num_items, D)
num_items, D = bert_embeddings.shape

# Create a Keras Embedding layer that simply returns bert_embeddings[i]
bert_embed_layer = tf.keras.layers.Embedding(
    input_dim=num_items,
    output_dim=D,
    embeddings_initializer=tf.keras.initializers.Constant(bert_embeddings),
    trainable=False,          # freeze so weights never change
    name="bert_embed_lookup"
)


In [29]:
from tensorflow.keras.layers import Input, Embedding, Flatten, Multiply, Concatenate, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import mixed_precision

mixed_precision.set_global_policy("mixed_float16")

K_mf   = 16
K_mlp  = 32
mlp_layers = [64, 32]   # just two hidden layers
dropout_rate = 0.2

# --- Model Definition ---
# 1) Define just integer inputs
user_in = Input(shape=(), dtype=tf.int32, name="user_input")
item_in = Input(shape=(), dtype=tf.int32, name="item_input")

# 2) GMF branch (unchanged)
u_mf = Embedding(num_users, K_mf, name="u_embed_mf")(user_in)
i_mf = Embedding(num_items, K_mf, name="i_embed_mf")(item_in)
u_latent_mf = Flatten()(u_mf)
i_latent_mf = Flatten()(i_mf)
mf_vec = Multiply()([u_latent_mf, i_latent_mf])  # (batch, K_mf)

# 3) MLP branch: learned user/item embeddings
u_mlp = Embedding(num_users, K_mlp, name="u_embed_mlp")(user_in)
i_mlp = Embedding(num_items, K_mlp, name="i_embed_mlp")(item_in)
u_latent_mlp = Flatten()(u_mlp)
i_latent_mlp = Flatten()(i_mlp)

# 4) **Here is where we “lookup” BERT features** for each item index
bert_feat = bert_embed_layer(item_in)  # shape = (batch_size, D)

# 5) Concatenate (u_latent_mlp, i_latent_mlp, bert_feat)
merged_mlp = Concatenate(name="mlp_input_concat")(
    [u_latent_mlp, i_latent_mlp, bert_feat]
)  # shape = (batch_size, 2*K_mlp + D)

# 6) Continue with your MLP layers
x = merged_mlp
for idx, hidden_size in enumerate(mlp_layers):
    x = Dense(hidden_size, activation="relu", name=f"mlp_dense_{idx}")(x)
    x = Dropout(dropout_rate)(x)

# 7) Final “fusion” with GMF part
final_concat = Concatenate()([mf_vec, x])
output = Dense(1, activation="sigmoid", name="prediction")(final_concat)

model_with_feats = Model(inputs=[user_in, item_in], outputs=output)
model_with_feats.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=[tf.keras.metrics.AUC(name="auc")]
)
model_with_feats.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_input          │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ item_input          │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ u_embed_mlp         │ (None, 32)        │  1,690,272 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ i_embed_mlp         │ (None, 32)        │    137,184 │ item_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_6 (Flatten) │ (None, 32)        │          0 │ u_embed_mlp[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_7 (Flatten) │ (None, 32)        │          0 │ i_embed_mlp[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bert_embed_lookup   │ (None, 768)       │  3,292,416 │ item_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_input_concat    │ (None, 832)       │          0 │ flatten_6[0][0],  │
│ (Concatenate)       │                   │            │ flatten_7[0][0],  │
│                     │                   │            │ bert_embed_looku… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_dense_0 (Dense) │ (None, 64)        │     53,312 │ mlp_input_concat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ u_embed_mf          │ (None, 16)        │    845,136 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ i_embed_mf          │ (None, 16)        │     68,592 │ item_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 64)        │          0 │ mlp_dense_0[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_4 (Flatten) │ (None, 16)        │          0 │ u_embed_mf[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_5 (Flatten) │ (None, 16)        │          0 │ i_embed_mf[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_dense_1 (Dense) │ (None, 32)        │      2,080 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_1          │ (None, 16)        │          0 │ flatten_4[0][0],  │
│ (Multiply)          │                   │            │ flatten_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 32)        │          0 │ mlp_dense_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 48)        │          0 │ multiply_1[0][0], │
│ (Concatenate)       │                   │            │ dropout_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ prediction (Dense)  │ (None, 1)         │         49 │ concatenate_2[0]… │
└─────────────────────┴───────────────────┴────────────┴─────────────────

 Total params: 6,089,041 (23.23 MB)

 Trainable params: 2,796,625 (10.67 MB)

 Non-trainable params: 3,292,416 (12.56 MB)

In [30]:
ks = [5, 10, 20]
rank_cb = MultiKRankingCallback(
    train_pos=train_pos,      # user → set of train-positive items
    test_pos=test_pos,        # user → set of test-positive items
    num_items=num_items,      # total number of distinct items (4287)
    Ks=ks,                     # compute Precision@10, NDCG@10, MAP@10
    verbose=1                 # set to 1 to print metrics each epoch
)

Train with extra embeddings

In [31]:
batch_size = 4096  # increase if your GPU/Colab can handle it (maybe 4096)
buffer_size = 100_000

# 1) Build a Dataset from your three 1D NumPy arrays
#    Each element is (u_idx, i_idx, label).
train_ds = tf.data.Dataset.from_tensor_slices((user_input, item_input, labels))

# 2) Shuffle → batch → prefetch.  Drop remainder so each batch is exactly `batch_size`.
train_ds = (
    train_ds
    .shuffle(buffer_size, seed=42)
    .batch(batch_size, drop_remainder=True)
    .map(lambda u, i, y: ((u, i), y), num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(tf.data.AUTOTUNE)
)

# 3) Do the same for validation (but typically you skip shuffle for val):
val_ds = tf.data.Dataset.from_tensor_slices(
    (user_input_test, item_input_test, labels_test)
)
val_ds = (
    val_ds
    .batch(batch_size, drop_remainder=False)
    .map(lambda u, i, y: ((u, i), y), num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(tf.data.AUTOTUNE)
)

# 4) Now call .fit on the Datasets:
history_with_feats = model_with_feats.fit(
    train_ds,
    epochs=5,
    validation_data=val_ds,
    callbacks=[rank_cb],
    shuffle=False  # we already shuffled in the Dataset pipeline
)


Epoch 1/5
2456/2456 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step - auc: 0.8334 - loss: 0.4595 — epoch 1:   P@5=0.110, NDCG@5=0.121, MAP@5=0.067, ILD@5=0.311, Nov@5=0.497;  P@10=0.095, NDCG@10=0.108, MAP@10=0.045, ILD@10=0.304, Nov@10=0.556;  P@20=0.078, NDCG@20=0.141, MAP@20=0.054, ILD@20=0.305, Nov@20=0.620;
2456/2456 ━━━━━━━━━━━━━━━━━━━━ 307s 124ms/step - auc: 0.8334 - loss: 0.4595 - val_auc: 0.9128 - val_loss: 0.3603 - Precision@5: 0.1100 - NDCG@5: 0.1208 - MAP@5: 0.0671 - ILD@5: 0.3107 - Novelty@5: 0.4974 - Precision@10: 0.0950 - NDCG@10: 0.1077 - MAP@10: 0.0446 - ILD@10: 0.3041 - Novelty@10: 0.5558 - Precision@20: 0.0780 - NDCG@20: 0.1411 - MAP@20: 0.0538 - ILD@20: 0.3046 - Novelty@20: 0.6202
Epoch 2/5
2456/2456 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - auc: 0.9102 - loss: 0.3552 — epoch 2:   P@5=0.098, NDCG@5=0.110, MAP@5=0.061, ILD@5=0.307, Nov@5=0.574;  P@10=0.107, NDCG@10=0.111, MAP@10=0.044, ILD@10=0.302, Nov@10=0.611;  P@20=0.099, NDCG@20=0.162, MAP@20=0.061, ILD@20=0.305, Nov@20=0.668;
24

# NCF + ResNet images

In [34]:
portrait_embeddings = np.load("resnet_book_embeddings.npy")  # (num_items, D)
num_items, D2 = portrait_embeddings.shape

portrait_embeddings.shape
print(f"num_items={num_items}, D={D2}")


num_items=4287, D=2048


In [35]:
import numpy as np

# Suppose portrait_embeddings is a NumPy array of shape (num_items, D)
portrait_embeddings = np.load("resnet_book_embeddings.npy")  # (num_items, D)
num_items, D = portrait_embeddings.shape

# Create a Keras Embedding layer that simply returns bert_embeddings[i]
resnet_embed_layer = tf.keras.layers.Embedding(
    input_dim=num_items,
    output_dim=D,
    embeddings_initializer=tf.keras.initializers.Constant(portrait_embeddings),
    trainable=False,          # freeze so weights never change
    name="resnet_embed_lookup"
)


In [36]:
from tensorflow.keras.layers import Input, Embedding, Flatten, Multiply, Concatenate, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import mixed_precision

mixed_precision.set_global_policy("mixed_float16")

K_mf   = 16
K_mlp  = 32
mlp_layers = [64, 32]   # just two hidden layers
dropout_rate = 0.2

# --- Model Definition ---
# 1) Define just integer inputs
user_in = Input(shape=(), dtype=tf.int32, name="user_input")
item_in = Input(shape=(), dtype=tf.int32, name="item_input")

# 2) GMF branch (unchanged)
u_mf = Embedding(num_users, K_mf, name="u_embed_mf")(user_in)
i_mf = Embedding(num_items, K_mf, name="i_embed_mf")(item_in)
u_latent_mf = Flatten()(u_mf)
i_latent_mf = Flatten()(i_mf)
mf_vec = Multiply()([u_latent_mf, i_latent_mf])  # (batch, K_mf)

# 3) MLP branch: learned user/item embeddings
u_mlp = Embedding(num_users, K_mlp, name="u_embed_mlp")(user_in)
i_mlp = Embedding(num_items, K_mlp, name="i_embed_mlp")(item_in)
u_latent_mlp = Flatten()(u_mlp)
i_latent_mlp = Flatten()(i_mlp)

# 4) **Here is where we “lookup” resnet features** for each item index
resnet_feat = resnet_embed_layer(item_in)  # shape = (batch_size, D)

# 5) Concatenate (u_latent_mlp, i_latent_mlp, bert_feat)
merged_mlp = Concatenate(name="mlp_input_concat")(
    [u_latent_mlp, i_latent_mlp, resnet_feat]
)  # shape = (batch_size, 2*K_mlp + D)

# 6) Continue with your MLP layers
x = merged_mlp
for idx, hidden_size in enumerate(mlp_layers):
    x = Dense(hidden_size, activation="relu", name=f"mlp_dense_{idx}")(x)
    x = Dropout(dropout_rate)(x)

# 7) Final “fusion” with GMF part
final_concat = Concatenate()([mf_vec, x])
output = Dense(1, activation="sigmoid", name="prediction")(final_concat)

model_with_feats_2 = Model(inputs=[user_in, item_in], outputs=output)
model_with_feats_2.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=[tf.keras.metrics.AUC(name="auc")]
)
model_with_feats_2.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_input          │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ item_input          │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ u_embed_mlp         │ (None, 32)        │  1,690,272 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ i_embed_mlp         │ (None, 32)        │    137,184 │ item_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_10          │ (None, 32)        │          0 │ u_embed_mlp[0][0] │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_11          │ (None, 32)        │          0 │ i_embed_mlp[0][0] │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ resnet_embed_lookup │ (None, 2048)      │  8,779,776 │ item_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_input_concat    │ (None, 2112)      │          0 │ flatten_10[0][0], │
│ (Concatenate)       │                   │            │ flatten_11[0][0], │
│                     │                   │            │ resnet_embed_loo… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_dense_0 (Dense) │ (None, 64)        │    135,232 │ mlp_input_concat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ u_embed_mf          │ (None, 16)        │    845,136 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ i_embed_mf          │ (None, 16)        │     68,592 │ item_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 64)        │          0 │ mlp_dense_0[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_8 (Flatten) │ (None, 16)        │          0 │ u_embed_mf[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_9 (Flatten) │ (None, 16)        │          0 │ i_embed_mf[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_dense_1 (Dense) │ (None, 32)        │      2,080 │ dropout_4[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_2          │ (None, 16)        │          0 │ flatten_8[0][0],  │
│ (Multiply)          │                   │            │ flatten_9[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 32)        │          0 │ mlp_dense_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 48)        │          0 │ multiply_2[0][0], │
│ (Concatenate)       │                   │            │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 11,658,321 (44.47 MB)

 Trainable params: 2,878,545 (10.98 MB)

 Non-trainable params: 8,779,776 (33.49 MB)

Training

In [37]:
ks = [5, 10, 20]
rank_cb = MultiKRankingCallback(
    train_pos=train_pos,      # user → set of train-positive items
    test_pos=test_pos,        # user → set of test-positive items
    num_items=num_items,      # total number of distinct items (4287)
    Ks=ks,                     # compute Precision@k, NDCG@k, MAP@k
    verbose=1                 # set to 1 to print metrics each epoch
)

In [38]:
batch_size = 4096  # increase if your GPU/Colab can handle it (maybe 4096)
buffer_size = 100_000

# 1) Build a Dataset from your three 1D NumPy arrays
#    Each element is (u_idx, i_idx, label).
train_ds = tf.data.Dataset.from_tensor_slices((user_input, item_input, labels))

# 2) Shuffle → batch → prefetch.  Drop remainder so each batch is exactly `batch_size`.
train_ds = (
    train_ds
    .shuffle(buffer_size, seed=42)
    .batch(batch_size, drop_remainder=True)
    .map(lambda u, i, y: ((u, i), y), num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(tf.data.AUTOTUNE)
)

# 3) Do the same for validation (but typically you skip shuffle for val):
val_ds = tf.data.Dataset.from_tensor_slices(
    (user_input_test, item_input_test, labels_test)
)
val_ds = (
    val_ds
    .batch(batch_size, drop_remainder=False)
    .map(lambda u, i, y: ((u, i), y), num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(tf.data.AUTOTUNE)
)

# 4) Now call .fit on the Datasets:
history_with_feats_2 = model_with_feats_2.fit(
    train_ds,
    epochs=5,
    validation_data=val_ds,
    callbacks=[rank_cb],
    shuffle=False  # we already shuffled in the Dataset pipeline
)


Epoch 1/5
2456/2456 ━━━━━━━━━━━━━━━━━━━━ 0s 246ms/step - auc: 0.8044 - loss: 0.4895 — epoch 1:   P@5=0.102, NDCG@5=0.112, MAP@5=0.060, ILD@5=0.314, Nov@5=0.550;  P@10=0.095, NDCG@10=0.103, MAP@10=0.040, ILD@10=0.308, Nov@10=0.596;  P@20=0.079, NDCG@20=0.139, MAP@20=0.052, ILD@20=0.305, Nov@20=0.649;
2456/2456 ━━━━━━━━━━━━━━━━━━━━ 628s 254ms/step - auc: 0.8044 - loss: 0.4895 - val_auc: 0.9105 - val_loss: 0.3618 - Precision@5: 0.1020 - NDCG@5: 0.1122 - MAP@5: 0.0604 - ILD@5: 0.3143 - Novelty@5: 0.5500 - Precision@10: 0.0950 - NDCG@10: 0.1034 - MAP@10: 0.0400 - ILD@10: 0.3083 - Novelty@10: 0.5960 - Precision@20: 0.0790 - NDCG@20: 0.1393 - MAP@20: 0.0520 - ILD@20: 0.3050 - Novelty@20: 0.6486
Epoch 2/5
2456/2456 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step - auc: 0.9054 - loss: 0.3629 — epoch 2:   P@5=0.126, NDCG@5=0.135, MAP@5=0.078, ILD@5=0.311, Nov@5=0.624;  P@10=0.105, NDCG@10=0.116, MAP@10=0.049, ILD@10=0.309, Nov@10=0.660;  P@20=0.098, NDCG@20=0.166, MAP@20=0.065, ILD@20=0.308, Nov@20=0.694;
24



# BERT + ResNet



In [39]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Embedding, Flatten, Multiply,
    Concatenate, Dense, Dropout
)
from tensorflow.keras.models import Model
from tensorflow.keras import mixed_precision

# 0) Optional: mixed-precision
mixed_precision.set_global_policy("mixed_float16")

# 1) Hyperparameters
K_mf       = 16
K_mlp      = 32
mlp_layers = [64, 32]
dropout    = 0.2

# Suppose:
#   bert_embed_layer   = Embedding(num_items, D1,
#                                embeddings_initializer=Constant(bert_embeddings), trainable=False)
#   resnet_embed_layer = Embedding(num_items, D2,
#                                embeddings_initializer=Constant(portrait_embeddings), trainable=False)

# 2) Inputs
user_in = Input(shape=(), dtype=tf.int32, name="user_input")
item_in = Input(shape=(), dtype=tf.int32, name="item_input")

# 3) GMF branch
u_mf = Embedding(num_users, K_mf, name="u_embed_mf")(user_in)
i_mf = Embedding(num_items, K_mf, name="i_embed_mf")(item_in)
u_lat_mf = Flatten()(u_mf)
i_lat_mf = Flatten()(i_mf)
mf_vec   = Multiply()([u_lat_mf, i_lat_mf])

# 4) MLP branch — learned embeddings
u_mlp = Embedding(num_users, K_mlp, name="u_embed_mlp")(user_in)
i_mlp = Embedding(num_items, K_mlp, name="i_embed_mlp")(item_in)
u_lat_mlp = Flatten()(u_mlp)
i_lat_mlp = Flatten()(i_mlp)

# 5) Look up both feature sets
bert_feat   = bert_embed_layer(item_in)    # shape=(batch, D1)
resnet_feat = resnet_embed_layer(item_in)  # shape=(batch, D2)

# 6) (Optional) Project each to smaller dimension, e.g. 64
bert_proj   = Dense(64, activation="relu", name="proj_bert")(bert_feat)
resnet_proj = Dense(64, activation="relu", name="proj_resnet")(resnet_feat)

# 7) Concatenate all MLP inputs
mlp_input = Concatenate(name="mlp_input")([
    u_lat_mlp,
    i_lat_mlp,
    bert_proj,
    resnet_proj
])  # shape = (batch, 2*K_mlp + 64 + 64)

# 8) MLP layers
x = mlp_input
for idx, size in enumerate(mlp_layers):
    x = Dense(size, activation="relu", name=f"mlp_dense_{idx}")(x)
    x = Dropout(dropout)(x)

# 9) Final fusion + output
final = Concatenate()([mf_vec, x])
out   = Dense(1, activation="sigmoid", name="prediction")(final)

model_both = Model(inputs=[user_in, item_in], outputs=out)
model_both.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=[tf.keras.metrics.AUC(name="auc")]
)
model_both.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_input          │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ item_input          │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ u_embed_mlp         │ (None, 32)        │  1,690,272 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ i_embed_mlp         │ (None, 32)        │    137,184 │ item_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bert_embed_lookup   │ (None, 768)       │  3,292,416 │ item_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ resnet_embed_lookup │ (None, 2048)      │  8,779,776 │ item_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_14          │ (None, 32)        │          0 │ u_embed_mlp[0][0] │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_15          │ (None, 32)        │          0 │ i_embed_mlp[0][0] │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ proj_bert (Dense)   │ (None, 64)        │     49,216 │ bert_embed_looku… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ proj_resnet (Dense) │ (None, 64)        │    131,136 │ resnet_embed_loo… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_input           │ (None, 192)       │          0 │ flatten_14[0][0], │
│ (Concatenate)       │                   │            │ flatten_15[0][0], │
│                     │                   │            │ proj_bert[0][0],  │
│                     │                   │            │ proj_resnet[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_dense_0 (Dense) │ (None, 64)        │     12,352 │ mlp_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ u_embed_mf          │ (None, 16)        │    845,136 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ i_embed_mf          │ (None, 16)        │     68,592 │ item_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_6 (Dropout) │ (None, 64)        │          0 │ mlp_dense_0[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_12          │ (None, 16)        │          0 │ u_embed_mf[0][0]  │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_13          │ (None, 16)        │          0 │ i_embed_mf[0][0]  │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 15,008,209 (57.25 MB)

 Trainable params: 2,936,017 (11.20 MB)

 Non-trainable params: 12,072,192 (46.05 MB)

Training

In [40]:
ks = [5, 10, 20]
rank_cb = MultiKRankingCallback(
    train_pos=train_pos,      # user → set of train-positive items
    test_pos=test_pos,        # user → set of test-positive items
    num_items=num_items,      # total number of distinct items (4287)
    Ks=ks,                     # compute Precision@k, NDCG@k, MAP@k
    verbose=1                 # set to 1 to print metrics each epoch
)

In [41]:
batch_size = 4096  # increase if your GPU/Colab can handle it (maybe 4096)
buffer_size = 100_000

# 1) Build a Dataset from your three 1D NumPy arrays
#    Each element is (u_idx, i_idx, label).
train_ds = tf.data.Dataset.from_tensor_slices((user_input, item_input, labels))

# 2) Shuffle → batch → prefetch.  Drop remainder so each batch is exactly `batch_size`.
train_ds = (
    train_ds
    .shuffle(buffer_size, seed=42)
    .batch(batch_size, drop_remainder=True)
    .map(lambda u, i, y: ((u, i), y), num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(tf.data.AUTOTUNE)
)

# 3) Do the same for validation (but typically you skip shuffle for val):
val_ds = tf.data.Dataset.from_tensor_slices(
    (user_input_test, item_input_test, labels_test)
)
val_ds = (
    val_ds
    .batch(batch_size, drop_remainder=False)
    .map(lambda u, i, y: ((u, i), y), num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(tf.data.AUTOTUNE)
)

# 4) Now call .fit on the Datasets:
history_both = model_both.fit(
    train_ds,
    epochs=5,
    validation_data=val_ds,
    callbacks=[rank_cb],
    shuffle=False  # we already shuffled in the Dataset pipeline
)


Epoch 1/5
2456/2456 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step - auc: 0.8282 - loss: 0.4639 — epoch 1:   P@5=0.104, NDCG@5=0.115, MAP@5=0.062, ILD@5=0.314, Nov@5=0.481;  P@10=0.088, NDCG@10=0.100, MAP@10=0.040, ILD@10=0.311, Nov@10=0.548;  P@20=0.086, NDCG@20=0.146, MAP@20=0.055, ILD@20=0.305, Nov@20=0.624;
2456/2456 ━━━━━━━━━━━━━━━━━━━━ 626s 253ms/step - auc: 0.8282 - loss: 0.4638 - val_auc: 0.9143 - val_loss: 0.3572 - Precision@5: 0.1040 - NDCG@5: 0.1147 - MAP@5: 0.0625 - ILD@5: 0.3144 - Novelty@5: 0.4815 - Precision@10: 0.0880 - NDCG@10: 0.1000 - MAP@10: 0.0396 - ILD@10: 0.3109 - Novelty@10: 0.5477 - Precision@20: 0.0855 - NDCG@20: 0.1465 - MAP@20: 0.0547 - ILD@20: 0.3048 - Novelty@20: 0.6237
Epoch 2/5
2456/2456 ━━━━━━━━━━━━━━━━━━━━ 0s 242ms/step - auc: 0.9128 - loss: 0.3504 — epoch 2:   P@5=0.114, NDCG@5=0.137, MAP@5=0.080, ILD@5=0.310, Nov@5=0.559;  P@10=0.105, NDCG@10=0.122, MAP@10=0.051, ILD@10=0.305, Nov@10=0.606;  P@20=0.097, NDCG@20=0.172, MAP@20=0.066, ILD@20=0.305, Nov@20=0.667;
24